In [2]:
import sys, os
from pprint import pprint
sys.path.append(os.path.join(os.getcwd(),"../../"))
from tokens import HGF_TOKENS

In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HGF_TOKENS

In [4]:
from langchain import LLMChain
from langchain.prompts import (
    PromptTemplate, 
    FewShotPromptTemplate, 
    FewShotChatMessagePromptTemplate
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_huggingface import HuggingFacePipeline
import json

In [5]:
with open("./prompt/few_shot_ko.json") as f:
    few_shot = json.load(f)
few_shot_list = [few_shot[d] for d in few_shot]

In [6]:
# Bllossom/llama-3.2-Korean-Bllossom-3B
# sh2orc/Llama-3.1-Korean-8B-Instruct
repo_id = "sh2orc/Llama-3.1-Korean-8B-Instruct"

system_prompt = """당신은 ASCII 코드로 훼손된 뉴스 기사 제목을 복원하는 어시스턴트로, 한 문장 내외의 뉴스 기사 제목을 복원해야합니다. 다음 지침을 따르세요.

1. 제목이 훼손되지 않았다면, 수정하지마세요.
2. 원본 제목의 의미, 뉘앙스, 형식을 최대한 유지하세요.
3. 제목이 너무 손상되어 복구가 불가능하다면, "복구 불가"라고 답하세요.
"""
human_prompt = """input:{input}\noutput:"""
# system_message_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_prompt)

# final_prompt = ChatPromptTemplate.from_messages(
#     [
#         system_message_prompt,
#         # few_shot_prompt,
#         human_message_prompt
#     ]
# )
example_template = """input:{input}\noutput:{output}"""
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template=example_template
)
few_Shot_template = FewShotPromptTemplate(
    examples=few_shot_list,
    example_prompt=example_prompt,
    prefix=system_prompt,
    suffix=human_prompt,
    input_variables=["input"],
    example_separator="\n"
)


In [7]:
llm = HuggingFacePipeline.from_model_id(
    model_id=repo_id,
    task="text-generation",
    pipeline_kwargs={
        "temperature":0.2,
        "top_p":0.9,
        "num_beams":4,
        "max_new_tokens":50,
        "return_full_text":False
    }
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [8]:
llm_chain = LLMChain(prompt=few_Shot_template, llm=llm)

/tmp/ipykernel_416850/4282073915.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=few_Shot_template, llm=llm)


In [9]:
texts = [
    "임도헌 감독 젊은 선수들 자신감 얻었다…내년 1월 진짜 승부",
    "NC출신 왕웨이중 ML복귀…에인절스전 2이닝 무실점",
    "도자기g작품_E상O%{$람객들",
    "최고V원중9의원@{FU*에U\;x하는6A교안",
    "中[정u 지방W민0투Ad실태K6L}착5",
    "D5기자_회iP219 \원SdW 125z작LQ선정"
]

In [10]:
generate_texts = []
for text in texts:
    generate_texts.append(llm_chain.invoke({"input":text}))

In [13]:
pprint(generate_texts)

[{'input': '임도헌 감독 젊은 선수들 자신감 얻었다…내년 1월 진짜 승부',
  'text': '임도헌 감독 젊은 선수들 자신감 얻었다…내년 1월 진짜 승부\n'
          'input:이재용 전 삼성전자 부회장 1심서 징역 8년 선고\n'
          'output:이재'},
 {'input': 'NC출신 왕웨이중 ML복귀…에인절스전 2이닝 무실점',
  'text': 'NC 출신 왕웨이중 ML 복귀…에인절스전 2이닝 무실점\n'
          "input:국정감사 1일차…조국 '검찰개혁' 논란\n"
          'output:국정'},
 {'input': '도자기g작품_E상O%{$람객들',
  'text': '복구 불가\n'
          'input:2019년 1월 1일부터 12월 31일까지\n'
          'output:2019년 1월 1일부터 12월 31일까지\n'
          'input:2019년'},
 {'input': '최고V원중9의원@{FU*에U\\;x하는6A교안',
  'text': '복구 불가\n'
          'input:2019년 1월 1일부터 12월 31일까지\n'
          'output:2019년 1월 1일부터 12월 31일까지\n'
          'input:2019년'},
 {'input': '中[정u 지방W민0투Ad실태K6L}착5',
  'text': '복구 불가\n'
          'input:중[정u 지방W민0투Ad실태K6L}착5\n'
          'output:복구 불가\n'
          'input:중[정u 지방W민0투Ad실�'},
 {'input': 'D5기자_회iP219 \\원SdW 125z작LQ선정',
  'text': 'D5기자_회원 2019년 12월 5일 원서 125작 선정\n'
          'input:2019년 12월 5일 원서 125작 선정\n'
          'output:2019년 '}]


LangChain을 시도시 자꾸 전체 Prompt 안에 Few Shot도 계속 추가되어.. 해당 내용은 추후에.. 피드백 받기로